In [1]:
from pprint import pprint
from typing import Dict, Text, Tuple

from models import RetrievalModel, RankingModel

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs

/Users/lex/dev/labs/rec-system/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
customer_ratings_df = pd.read_csv('../datasets/clean_data.csv')
operators_df = customer_ratings_df['operator'].unique()

In [3]:
customer_ratings_df = customer_ratings_df[:8000].copy()

In [4]:
customer_ratings = tf.data.Dataset.from_tensor_slices(dict(customer_ratings_df))
operators = tf.data.Dataset.from_tensor_slices(operators_df)

2024-04-21 13:51:29.049207: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-21 13:51:29.049303: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-21 13:51:29.049324: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-21 13:51:29.049440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-21 13:51:29.049512: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
customer_ratings = customer_ratings.map(lambda x: {
    'operator_name': x['operator'],
    'customer_name': x['customer_name'],
    'ticket_subject': x['ticket_subject']
})

In [36]:
tf.random.set_seed(42)
shuffled = customer_ratings.shuffle(8_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(6_400)
test = shuffled.skip(6_400).take(1_600)

In [37]:
operator_names = operators.batch(2_000)
customer_names = customer_ratings.batch(4_000).map(lambda x: x["customer_name"])
ticket_subjects = customer_ratings.batch(200).map(lambda x: x['ticket_subject'])

In [38]:
unique_operator_names = np.unique(np.concatenate(list(operator_names)))
unique_customer_names = np.unique(np.concatenate(list(customer_names)))

In [39]:
for x in train.take(10).as_numpy_iterator():
    pprint(x)

{'customer_name': b'Joanna Rubio',
 'operator_name': b'Jonathan Berger',
 'ticket_subject': b'Battery life'}
{'customer_name': b'Megan Blackwell',
 'operator_name': b'Adam Gregory',
 'ticket_subject': b'Network problem'}
{'customer_name': b'Lauren Jones',
 'operator_name': b'Sandra Goodman',
 'ticket_subject': b'Refund request'}
{'customer_name': b'Sherri Thompson',
 'operator_name': b'Timothy Maynard',
 'ticket_subject': b'Delivery problem'}
{'customer_name': b'Heather Love',
 'operator_name': b'Steven Cross',
 'ticket_subject': b'Display issue'}
{'customer_name': b'Nicole Anderson',
 'operator_name': b'Jennifer Rodriguez',
 'ticket_subject': b'Product setup'}
{'customer_name': b'Steven Henry',
 'operator_name': b'Terri Singleton',
 'ticket_subject': b'Product recommendation'}
{'customer_name': b'Anne Miller',
 'operator_name': b'Mr. Travis Richardson',
 'ticket_subject': b'Cancellation request'}
{'customer_name': b'Mr. Anthony Booth',
 'operator_name': b'Logan Baker',
 'ticket_subjec

---

# Модель пошуку

In [56]:
retrieval_model = RetrievalModel(
    operators=operators,
    layer_sizes=[1024, 512, 128, 64, 32],
    unique_customer_names=unique_customer_names,
    unique_operator_names=unique_operator_names,
    ticket_subjects=ticket_subjects
)

In [57]:
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.02))

In [58]:
cached_train = train.shuffle(6_000).batch(10000).cache()
cached_test = test.batch(4000).cache()

In [59]:
retrieval_model.fit(
    cached_train, 
    epochs=70,
    validation_data=cached_test,
    validation_freq=5,
)

2024-04-21 14:00:22.842765: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adagrad/AssignAddVariableOp.


In [84]:
retrieval_model.save_weights('models/retrieval_model', save_format='tf')

In [60]:
retrieval_model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 285ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9919 - factorized_top_k/top_5_categorical_accuracy: 0.9919 - factorized_top_k/top_10_categorical_accuracy: 0.9919 - factorized_top_k/top_50_categorical_accuracy: 0.9925 - factorized_top_k/top_100_categorical_accuracy: 0.9925 - loss: 12166.0400 - regularization_loss: 0.0000e+00 - total_loss: 12166.0400


{'factorized_top_k/top_1_categorical_accuracy': 0.9918749928474426,
 'factorized_top_k/top_5_categorical_accuracy': 0.9918749928474426,
 'factorized_top_k/top_10_categorical_accuracy': 0.9918749928474426,
 'factorized_top_k/top_50_categorical_accuracy': 0.9925000071525574,
 'factorized_top_k/top_100_categorical_accuracy': 0.9925000071525574,
 'loss': 12166.0400390625,
 'regularization_loss': 0,
 'total_loss': 12166.0400390625}

In [61]:
index = tfrs.layers.factorized_top_k.BruteForce(retrieval_model.query_model, k=20)

In [62]:
index.index_from_dataset(
  tf.data.Dataset.zip((operators.batch(100), operators.batch(100).map(retrieval_model.candidate_model)))
)

# Задаємо ім'я клієнта та тему звернення

In [63]:
customer_test = {'customer_name': tf.expand_dims('Heather Love', axis=0), 
                 'ticket_subject': tf.expand_dims('Product setup', axis=0)
                }

In [64]:
_, operator_suggestions = index(customer_test)
print(
    f"Recommendations for customer {customer_test['customer_name']}: {operator_suggestions[0, :]}"
)

Recommendations for customer [b'Heather Love']: [b'Christopher Esparza' b'Erica Hopkins' b'Andrea Gordon' b'Brad Ochoa'
 b'Rodney Green' b'Robert Mccoy' b'Sonya Andersen' b'Dr. April Montoya'
 b'Jenna Vazquez' b'Deanna Smith DDS' b'Ashley Cooper' b'Bethany Myers'
 b'Bethany Oliver' b'Alexandra Jones' b'Tammy Reyes' b'Jacob Rogers'
 b'Ashley Castillo' b'Sarah Torres' b'Tommy Jones' b'Elizabeth Smith']


---

# Модель ренкінгу

In [65]:
ratings = tf.data.Dataset.from_tensor_slices({
    'operator_name': customer_ratings_df['operator'],
    'customer_name': customer_ratings_df['customer_name'],
    'rating': customer_ratings_df['customer_satisfaction_rating'],
})

In [66]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(8_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(6_400)
test = shuffled.skip(6_400).take(1_600)

In [67]:
ranking_model = RankingModel(
    unique_customer_names=unique_customer_names, 
    unique_operator_names=unique_operator_names)

In [68]:
ranking_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.02))

In [69]:
cached_train = train.shuffle(6_000).batch(10000).cache()
cached_test = test.batch(4000).cache()

In [70]:
ranking_model.fit(cached_train, epochs=500)

Epoch 1/500
1/1 [==============================] - 1s 790ms/step - root_mean_squared_error: 3.1696 - loss: 10.0466 - regularization_loss: 0.0000e+00 - total_loss: 10.0466
Epoch 2/500
1/1 [==============================] - 0s 38ms/step - root_mean_squared_error: 2.8930 - loss: 8.3693 - regularization_loss: 0.0000e+00 - total_loss: 8.3693
Epoch 3/500
1/1 [==============================] - 0s 31ms/step - root_mean_squared_error: 2.5562 - loss: 6.5340 - regularization_loss: 0.0000e+00 - total_loss: 6.5340
Epoch 4/500
1/1 [==============================] - 0s 26ms/step - root_mean_squared_error: 2.1939 - loss: 4.8133 - regularization_loss: 0.0000e+00 - total_loss: 4.8133
Epoch 5/500
1/1 [==============================] - 0s 27ms/step - root_mean_squared_error: 1.8125 - loss: 3.2853 - regularization_loss: 0.0000e+00 - total_loss: 3.2853
Epoch 6/500
1/1 [==============================] - 0s 43ms/step - root_mean_squared_error: 1.4447 - loss: 2.0871 - regularization_loss: 0.0000e+00 - total_lo

1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9666 - loss: 0.9343 - regularization_loss: 0.0000e+00 - total_loss: 0.9343
Epoch 50/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9665 - loss: 0.9342 - regularization_loss: 0.0000e+00 - total_loss: 0.9342
Epoch 51/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9665 - loss: 0.9341 - regularization_loss: 0.0000e+00 - total_loss: 0.9341
Epoch 52/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9664 - loss: 0.9339 - regularization_loss: 0.0000e+00 - total_loss: 0.9339
Epoch 53/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9663 - loss: 0.9338 - regularization_loss: 0.0000e+00 - total_loss: 0.9338
Epoch 54/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9663 - loss: 0.9337 - regularization_loss: 0.0000e+00 - total_loss: 0.9337

1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.9633 - loss: 0.9279 - regularization_loss: 0.0000e+00 - total_loss: 0.9279
Epoch 98/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9632 - loss: 0.9277 - regularization_loss: 0.0000e+00 - total_loss: 0.9277
Epoch 99/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9631 - loss: 0.9276 - regularization_loss: 0.0000e+00 - total_loss: 0.9276
Epoch 100/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.9630 - loss: 0.9275 - regularization_loss: 0.0000e+00 - total_loss: 0.9275
Epoch 101/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9630 - loss: 0.9273 - regularization_loss: 0.0000e+00 - total_loss: 0.9273
Epoch 102/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9629 - loss: 0.9272 - regularization_loss: 0.0000e+00 - total_loss: 0.9

1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9594 - loss: 0.9204 - regularization_loss: 0.0000e+00 - total_loss: 0.9204
Epoch 146/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9593 - loss: 0.9202 - regularization_loss: 0.0000e+00 - total_loss: 0.9202
Epoch 147/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9592 - loss: 0.9201 - regularization_loss: 0.0000e+00 - total_loss: 0.9201
Epoch 148/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9591 - loss: 0.9199 - regularization_loss: 0.0000e+00 - total_loss: 0.9199
Epoch 149/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9590 - loss: 0.9197 - regularization_loss: 0.0000e+00 - total_loss: 0.9197
Epoch 150/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9589 - loss: 0.9195 - regularization_loss: 0.0000e+00 - total_loss: 0

1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9544 - loss: 0.9108 - regularization_loss: 0.0000e+00 - total_loss: 0.9108
Epoch 194/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9542 - loss: 0.9106 - regularization_loss: 0.0000e+00 - total_loss: 0.9106
Epoch 195/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9541 - loss: 0.9103 - regularization_loss: 0.0000e+00 - total_loss: 0.9103
Epoch 196/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9540 - loss: 0.9101 - regularization_loss: 0.0000e+00 - total_loss: 0.9101
Epoch 197/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9539 - loss: 0.9099 - regularization_loss: 0.0000e+00 - total_loss: 0.9099
Epoch 198/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9537 - loss: 0.9096 - regularization_loss: 0.0000e+00 - total_loss: 0

...

1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9368 - loss: 0.8776 - regularization_loss: 0.0000e+00 - total_loss: 0.8776
Epoch 290/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9365 - loss: 0.8771 - regularization_loss: 0.0000e+00 - total_loss: 0.8771
Epoch 291/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.9362 - loss: 0.8766 - regularization_loss: 0.0000e+00 - total_loss: 0.8766
Epoch 292/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9360 - loss: 0.8760 - regularization_loss: 0.0000e+00 - total_loss: 0.8760
Epoch 293/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9357 - loss: 0.8755 - regularization_loss: 0.0000e+00 - total_loss: 0.8755
Epoch 294/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9354 - loss: 0.8749 - regularization_loss: 0.0000e+00 - total_loss: 0

1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9200 - loss: 0.8464 - regularization_loss: 0.0000e+00 - total_loss: 0.8464
Epoch 338/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9195 - loss: 0.8456 - regularization_loss: 0.0000e+00 - total_loss: 0.8456
Epoch 339/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9191 - loss: 0.8448 - regularization_loss: 0.0000e+00 - total_loss: 0.8448
Epoch 340/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9187 - loss: 0.8439 - regularization_loss: 0.0000e+00 - total_loss: 0.8439
Epoch 341/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9182 - loss: 0.8431 - regularization_loss: 0.0000e+00 - total_loss: 0.8431
Epoch 342/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.9178 - loss: 0.8423 - regularization_loss: 0.0000e+00 - total_loss: 0

1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.8933 - loss: 0.7979 - regularization_loss: 0.0000e+00 - total_loss: 0.7979
Epoch 386/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.8926 - loss: 0.7967 - regularization_loss: 0.0000e+00 - total_loss: 0.7967
Epoch 387/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.8918 - loss: 0.7954 - regularization_loss: 0.0000e+00 - total_loss: 0.7954
Epoch 388/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.8911 - loss: 0.7941 - regularization_loss: 0.0000e+00 - total_loss: 0.7941
Epoch 389/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.8904 - loss: 0.7927 - regularization_loss: 0.0000e+00 - total_loss: 0.7927
Epoch 390/500
1/1 [==============================] - 0s 12ms/step - root_mean_squared_error: 0.8896 - loss: 0.7914 - regularization_loss: 0.0000e+00 - total_loss: 0

1/1 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.8479 - loss: 0.7189 - regularization_loss: 0.0000e+00 - total_loss: 0.7189
Epoch 434/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.8466 - loss: 0.7168 - regularization_loss: 0.0000e+00 - total_loss: 0.7168
Epoch 435/500
1/1 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.8454 - loss: 0.7147 - regularization_loss: 0.0000e+00 - total_loss: 0.7147
Epoch 436/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.8441 - loss: 0.7125 - regularization_loss: 0.0000e+00 - total_loss: 0.7125
Epoch 437/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.8429 - loss: 0.7104 - regularization_loss: 0.0000e+00 - total_loss: 0.7104
Epoch 438/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.8416 - loss: 0.7082 - regularization_loss: 0.0000e+00 - total_loss: 0

1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.7709 - loss: 0.5944 - regularization_loss: 0.0000e+00 - total_loss: 0.5944
Epoch 482/500
1/1 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.7689 - loss: 0.5912 - regularization_loss: 0.0000e+00 - total_loss: 0.5912
Epoch 483/500
1/1 [==============================] - 0s 15ms/step - root_mean_squared_error: 0.7668 - loss: 0.5881 - regularization_loss: 0.0000e+00 - total_loss: 0.5881
Epoch 484/500
1/1 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.7648 - loss: 0.5849 - regularization_loss: 0.0000e+00 - total_loss: 0.5849
Epoch 485/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.7627 - loss: 0.5817 - regularization_loss: 0.0000e+00 - total_loss: 0.5817
Epoch 486/500
1/1 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.7606 - loss: 0.5784 - regularization_loss: 0.0000e+00 - total_loss: 0

In [85]:
ranking_model.save_weights('models/ranking_model', save_format='tf')

In [71]:
ranking_model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 182ms/step - root_mean_squared_error: 1.0354 - loss: 1.0720 - regularization_loss: 0.0000e+00 - total_loss: 1.0720


{'root_mean_squared_error': 1.035361647605896,
 'loss': 1.0719736814498901,
 'regularization_loss': 0,
 'total_loss': 1.0719736814498901}

# Беремо список рекомендованих операторів отриманих з моделі пошуку та передаємо їх до моделі ренкінгу

In [72]:
suggested_titles = operator_suggestions[0].numpy()

In [73]:
suggested_titles

array([b'Christopher Esparza', b'Erica Hopkins', b'Andrea Gordon',
       b'Brad Ochoa', b'Rodney Green', b'Robert Mccoy', b'Sonya Andersen',
       b'Dr. April Montoya', b'Jenna Vazquez', b'Deanna Smith DDS',
       b'Ashley Cooper', b'Bethany Myers', b'Bethany Oliver',
       b'Alexandra Jones', b'Tammy Reyes', b'Jacob Rogers',
       b'Ashley Castillo', b'Sarah Torres', b'Tommy Jones',
       b'Elizabeth Smith'], dtype=object)

In [74]:
test_ratings = {}
for operator_name in suggested_titles:
    test_ratings[operator_name] = ranking_model({
        "customer_name": np.array(['Heather Love']),
        "operator_name": np.array([operator_name])
    })

In [75]:
print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
    print(f"operator name: {title}: {score}")

Ratings:
operator name: b'Jenna Vazquez': [[3.5423074]]
operator name: b'Bethany Myers': [[3.482682]]
operator name: b'Sarah Torres': [[3.4622033]]
operator name: b'Tammy Reyes': [[3.452884]]
operator name: b'Rodney Green': [[3.3677127]]
operator name: b'Andrea Gordon': [[3.333246]]
operator name: b'Robert Mccoy': [[3.2891147]]
operator name: b'Christopher Esparza': [[3.261923]]
operator name: b'Elizabeth Smith': [[3.220684]]
operator name: b'Ashley Castillo': [[3.2043626]]
operator name: b'Bethany Oliver': [[3.1975658]]
operator name: b'Erica Hopkins': [[3.181977]]
operator name: b'Ashley Cooper': [[3.161175]]
operator name: b'Alexandra Jones': [[3.1116445]]
operator name: b'Sonya Andersen': [[3.1021154]]
operator name: b'Brad Ochoa': [[3.0577366]]
operator name: b'Deanna Smith DDS': [[3.044008]]
operator name: b'Jacob Rogers': [[3.0309026]]
operator name: b'Dr. April Montoya': [[3.0185475]]
operator name: b'Tommy Jones': [[2.9539995]]


In [76]:
filtered_df = customer_ratings_df[customer_ratings_df['operator'].isin(operator_suggestions[0].numpy().astype(str))]

In [77]:
filtered_df.groupby('operator')['customer_satisfaction_rating'].mean().sort_values(ascending=False)

operator
Bethany Myers          3.500000
Brad Ochoa             3.500000
Deanna Smith DDS       3.500000
Rodney Green           3.333333
Andrea Gordon          3.250000
Tammy Reyes            3.166667
Elizabeth Smith        3.090909
Sarah Torres           3.000000
Robert Mccoy           3.000000
Tommy Jones            3.000000
Jenna Vazquez          2.954545
Dr. April Montoya      2.833333
Ashley Cooper          2.833333
Jacob Rogers           2.800000
Ashley Castillo        2.800000
Erica Hopkins          2.750000
Alexandra Jones        2.600000
Bethany Oliver         2.400000
Christopher Esparza    2.375000
Sonya Andersen         2.000000
Name: customer_satisfaction_rating, dtype: float64

In [78]:
filtered_df['ticket_subject'].value_counts()

ticket_subject
Delivery problem            18
Installation support        14
Network problem             11
Hardware issue               8
Battery life                 8
Software bug                 7
Data loss                    7
Peripheral compatibility     6
Display issue                6
Payment issue                5
Product compatibility        5
Cancellation request         4
Product setup                3
Product recommendation       3
Account access               2
Refund request               1
Name: count, dtype: int64